In [1]:
from __future__ import unicode_literals
import pandas as pd 
import numpy as np
import os 

from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn import preprocessing

from string import punctuation
from collections import Counter , defaultdict

from sklearn import svm, datasets
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit

import gensim
import struct
from gensim.models.word2vec import Word2Vec

In [3]:
data = pd.read_csv('review.csv')

In [4]:
DataX = data['review']
DataY = data['sentiment']
y = DataY

In [5]:
data.head()

,review,sentiment
0,"lancelot du lac ( lancelot du lac ) ( france ,...",1
1,director : brian de palma writer : david koepp...,1
2,"six days , seven nights reviewed by jamie peck...",1
3,"cast : mel gibson ( jerry fletcher ) , julia r...",1
4,"all great things come to an end , and the dot-...",1


In [6]:
def String_Splitter(data):
    """
    
    """
    for i in range(len(data)):
        try :             
            data[i]=data[i].split()
        except:
            print(i)
            data[i] = data[i]
    return data




def wordToString(wordList):
    """
    
    """
    stringList = []
    for i in range(0,len(wordList)):
        stringList.append(' '.join(word for word in wordList[i]))
    return stringList




def TextCleaner(Data, stopwordsList= ''):
    """
    
    """
    stemmer = SnowballStemmer("english")
    lemmatizer = WordNetLemmatizer()
    dataList = Data
    table = str.maketrans('', '', punctuation)
    CountVector = []
    
    for i in range(0, len(dataList)):
        vocabulary = []
        
        for j in range(0, len(dataList[i])):
            dataList[i][j] = stemmer.stem(dataList[i][j])
            dataList[i][j] = lemmatizer.lemmatize(dataList[i][j])
            
        dataList[i] = [word for word in dataList[i] if word.isalpha()]
        dataList[i] = [w.translate(table) for w in dataList[i]]
        dataList[i] = [word for word in dataList[i] if len(word) > 3]
        vocabulary.append(dataList[i])
        
    return dataList

In [7]:
temp_corpus = DataX.values.tolist()

i = len(temp_corpus)-1
while i != 0:
    if temp_corpus[i]==float('nan'):
        del(temp_corpus[i])
    i-=1
    

new_data = String_Splitter(temp_corpus)
X = TextCleaner(new_data)

In [8]:
X[:1]

[['lancelot',
  'lancelot',
  'franc',
  'film',
  'review',
  'jame',
  'kendrick',
  'director',
  'robert',
  'bresson',
  'screenwrit',
  'robert',
  'bresson',
  'base',
  'epic',
  'poem',
  'chretien',
  'troy',
  'star',
  'simon',
  'lancelot',
  'laura',
  'duke',
  'condomina',
  'queen',
  'guinever',
  'vladimir',
  'king',
  'arthur',
  'humbert',
  'balsan',
  'gawain',
  'patrick',
  'bernhard',
  'modr',
  'arthur',
  'montalembert',
  'lionel',
  'mpaa',
  'rate',
  'rate',
  'review',
  'ench',
  'director',
  'robert',
  'bresson',
  'lancelot',
  'lake',
  'achiev',
  'exact',
  'opposit',
  'effect',
  'thoma',
  'malori',
  'mort',
  'darthur',
  'instead',
  'enshrin',
  'legend',
  'king',
  'arthur',
  'knight',
  'round',
  'tabl',
  'dethron',
  'them',
  'reveal',
  'arthur',
  'weak',
  'ineffectu',
  'leader',
  'knight',
  'group',
  'jealous',
  'bicker',
  'fail',
  'live',
  'legend',
  'prescrib',
  'them',
  'chivalri',
  'place',
  'lancelot',
  'l

In [9]:
whole_corpora = ""
for i in range(len(data)): 
    whole_corpora = whole_corpora + " " + data.iloc[i]['review'] 

In [11]:
word_embedding_path = "cc.en.300.vec"
encoding = "utf-8"
with open(word_embedding_path, "rb") as lines:
    wvec = {line.split()[0].decode(encoding): np.array(line.split()[1:],dtype=np.float32)
               for line in lines}